In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import networkit as nk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random
import phate

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances
from IPython.display import display, Math, Latex, Markdown
from tqdm.notebook import tqdm

from External.ICT.calculate_ICT import calculate_ICT
from External.clustering import centers, k_means_pp, ICT_clusters, cfilter, cassign
from External.generation import create_graph, find_backbone
from External.plotting import plot_points, plot_graph, no_intersections
from External.reweight import reweight_clusters
from External.data_loader import data_loader

from copy import deepcopy

# from External.create_k_nearest import patch_together

plt.style.use('standard.mplstyle')

In [5]:
# Hyperparameters
mode = "K_Nearest+Density2+Recompute" # Graph construction mode
ICT_algorithm = "cluster_all"
metric = "euclidean" # metric for clustering

# Cluster rassignment
min_cluster_size = 12

In [6]:
position, labels, embedding = data_loader("eth_CC", True)
dtae_embedding, dtae_labels = data_loader("CC_DTAE_embedding", False)

number_of_nodes = len(position)

# calculate the clusters
pca = PCA(n_components=100).fit_transform(StandardScaler().fit_transform(position))

In [ ]:
G, _ = create_graph(number_of_nodes, mode, position=pca, beta=2, delta=0.5)
G.indexEdges()

Final k: 11


calculating the densities for the density criterion:   0%|          | 0/13707 [00:00<?, ?it/s]

updating the distance array:   0%|          | 0/13707 [00:00<?, ?it/s]

In [ ]:
cluster_centers2 = centers(pca)

ICT = calculate_ICT(G, algorithm_type=ICT_algorithm, cluster_centers=cluster_centers2, zeros_stay_zeros=True, update_G=1.1)

ICT.indexEdges()

# Plot the ICT
fig, axs = plt.subplots(2, 2)

plot_graph(G, embedding, f"Cluster Centers", axs[0,0], cluster_centers=cluster_centers2, node_size=0)


plot_points(embedding, f"", axs[0,1], labels=labels, node_size=0.5)
plot_graph(ICT, embedding, f"Resulting ICT without nodes", axs[0,1], node_size=0)


plot_graph(G, dtae_embedding, f"Cluster Centers", axs[1,0], cluster_centers=cluster_centers2, node_size=0)


plot_points(dtae_embedding, f"", axs[1,1], labels=dtae_labels, node_size=0.5)
plot_graph(ICT, dtae_embedding, f"Resulting ICT without nodes", axs[1,1], node_size=0)

axs[1,0].set_xlim([-50,60])
axs[1,0].set_ylim([-45,50])
axs[1,1].set_xlim([-50,60])
axs[1,1].set_ylim([-45,50])

plt.tight_layout()
# plt.savefig(f"Output/ETH/pca/" + name.zfill(5) + ".png")
plt.show()

In [ ]:
fig, ax = plt.subplots(1)
no_intersections(ICT, position, ax=ax, labels=labels)

In [ ]:
fig, ax = plt.subplots(1)
backbone, backbone_nodes = find_backbone(ICT, number_of_vertices=30)
backbone.indexEdges()
plot_graph(backbone, embedding, f"backbone", ax, node_size=0)
plot_points(embedding, f"", ax, labels=labels, node_size=1)
plot_points(embedding[backbone_nodes], f"", ax, labels=labels[backbone_nodes], node_size=20)